In [1]:
#import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
from datetime import datetime as dt
from datetime import timedelta as delta
import numpy as np
import seaborn as sb
import os

ModuleNotFoundError: No module named 'yfinance'

In [10]:
signals = pd.DataFrame({
    '10_cross_30':[0,0,1,1,1],
    'MACD_Signal_MACD':[1,1,1,0,0],
    'MACD_lim':[0,0,0,1,1],
    'abv_50':[1,1,1,0,0],
    'abv_200':[0,1,0,0,1],
    'strategy': [1,2,3,4,5],
})
#print(signals)

In [11]:
def add_signal_indicators(df):
    df['SMA_10'] = ta.sma(df['Adj Close'],length=10)
    df['SMA_30'] = ta.sma(df['Adj Close'],length=30)
    df['SMA_50'] = ta.sma(df['Adj Close'],length=50)
    df['SMA_200'] = ta.sma(df['Adj Close'],length=200)
    
    macd = ta.macd(df['Adj Close'], fast=12, slow=26, signal=9)
    df['MACD'] = macd['MACD_12_26_9']
    df['MACD_signal'] = macd['MACDs_12_26_9']
    df['MACD_hist'] = macd['MACDh_12_26_9']

    df['10_cross_30'] = np.where(df['SMA_10'] > df['SMA_30'], 1, 0)
    
    df['MACD_Signal_MACD'] = np.where(df['MACD_signal'] < df['MACD'], 1, 0)
    
    df['MACD_lim'] = np.where(df['MACD']>0, 1, 0)
    
    df['abv_50'] = np.where((df['SMA_30']>df['SMA_50'])
                            &(df['SMA_10']>df['SMA_50']), 1, 0)
                            
    df['abv_200'] = np.where((df['SMA_30']>df['SMA_200'])
                            &(df['SMA_10']>df['SMA_200'])
                            &(df['SMA_50']>df['SMA_200']), 1, 0)
    
    return df

In [5]:
nifty_50_stocks = ['EICHERMOT','HEROMOTOCO','NESTLEIND','ONGC',
                   'BAJAJ-AUTO','TATASTEEL','GRASIM',
                   'BRITANNIA','BAJFINANCE','DIVISLAB','M&M',
                   'HINDUNILVR','HDFCBANK','HDFCLIFE','BHARTIARTL','TCS',
                   'LT','DRREDDY','ULTRACEMCO','SUNPHARMA','NTPC',
                   'TATAMOTORS','UPL','SBIN','HINDALCO','ITC','JSWSTEEL',
                   'COALINDIA','RELIANCE','BPCL','LTIM','MARUTI','HCLTECH',
                   'POWERGRID','WIPRO','SBILIFE','AXISBANK',
                   'ADANIPORTS','ICICIBANK','TITAN','BAJAJFINSV','KOTAKBANK',
                   'TATACONSUM','APOLLOHOSP','INFY','ASIANPAINT',
                   'ADANIENT','INDUSINDBK','TECHM','CIPLA']

In [5]:
nifty_50_stocks = ['TECHM','CIPLA']

In [21]:
def backtest_signals(row):
    global TRADES, trade_in_progress, signals, holding_period
    
    if(trade_in_progress):
        _data = TRADES[-1]
        # time to sell after n holding days
        if(row['index']==_data['sell_index']):
            _data['sell_price'] = round(row['Adj Close'],2)
            _data['sell_date'] = str(pd.to_datetime(row['Date']).date())
            _data['returns'] = round((_data['sell_price']-_data['buy_price'])/_data['buy_price']*100, 3)
            TRADES[-1] = _data
            trade_in_progress = False
            
    else:
        _r = pd.DataFrame([row])
        print(_r)
        _r = _r.merge(signals, on=list(signals.columns[:-1]), how='inner')
        strategy = _r.shape[0]
        
        if(strategy>0): 
            trade_in_progress = True
            TRADES.append({
                'strategy': _r['strategy'].values[0],
                'buy_date': str(pd.to_datetime(row['Date']).date()),
                'buy_index': row['index'],
                'sell_date': '',
                'sell_index': row['index'] + holding_period,
                'buy_price': round(row['Adj Close'], 2),
                'sell_price': '',
                'returns': 0,
                'stock': row['stock']
            })

In [ ]:
TRADES = []
trade_in_progress = False
holding_period = 14

for i in nifty_50_stocks:
    _df = yf.download(f"{i}.NS", 
                      period='1d', 
                      start='2022-01-01', 
                      end=(dt.now() + delta(1)).strftime('%Y-%m-%d'), 
                      progress=False)

    _df = add_signal_indicators(_df)

    _df['stock'] = i
    _df.reset_index().reset_index().apply(backtest_signals, axis=1)
    print(f'Done performing backtesting for {i}')
    del _df
    trade_in_progress = False

In [27]:
print(len(TRADES))
TRADES[0:40]

7


[{'strategy': 4,
  'buy_date': '2022-02-18',
  'buy_index': 33,
  'sell_date': '2022-03-11',
  'sell_index': 47,
  'buy_price': 2550.44,
  'sell_price': 2190.67,
  'returns': -14.106,
  'stock': 'HEROMOTOCO'},
 {'strategy': 5,
  'buy_date': '2022-11-24',
  'buy_index': 221,
  'sell_date': '2022-12-14',
  'sell_index': 235,
  'buy_price': 2576.16,
  'sell_price': 2647.71,
  'returns': 2.777,
  'stock': 'HEROMOTOCO'},
 {'strategy': 5,
  'buy_date': '2023-01-30',
  'buy_index': 267,
  'sell_date': '2023-02-17',
  'sell_index': 281,
  'buy_price': 2604.3,
  'sell_price': 2507.34,
  'returns': -3.723,
  'stock': 'HEROMOTOCO'},
 {'strategy': 5,
  'buy_date': '2023-09-28',
  'buy_index': 431,
  'sell_date': '2023-10-19',
  'sell_index': 445,
  'buy_price': 2970.0,
  'sell_price': 3265.85,
  'returns': 9.961,
  'stock': 'HEROMOTOCO'},
 {'strategy': 5,
  'buy_date': '2022-12-26',
  'buy_index': 243,
  'sell_date': '2023-01-13',
  'sell_index': 257,
  'buy_price': 19684.0,
  'sell_price': 19572.

In [ ]:
import matplotlib.pyplot as plt
pos = pd.DataFrame(TRADES).groupby('stock')['returns'].agg(['mean']).reset_index()
pos = pos.sort_values(by='mean', ascending=False).head(10)
ax = sb.barplot(x='stock', y='mean', data=pos)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.show()

In [ ]:
pos = pd.DataFrame(TRADES).groupby('stock')['returns'].agg(['mean']).reset_index()
pos = pos.sort_values(by='mean', ascending=False).head(11).tail(10)

ax = sb.barplot(x='stock', y='mean', data=pos)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.show()

In [ ]:
pos = pd.DataFrame(TRADES).groupby('stock')['returns'].agg(['mean']).reset_index()
pos = pos.sort_values(by='mean', ascending=True).head(10)
ax = sb.barplot(x='stock', y='mean', data=pos)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.show()

In [ ]:
from yahooquery import Ticker
stock = Ticker("SBIN.NS")
_df = stock.history(interval='1d', 
                    start='2022-01-01', 
                    end=(dt.now() + delta(1)).strftime('%Y-%m-%d'))
print(_df)
#_df = add_signal_indicators(_df)
#print(_df)

In [ ]:
import pandas as pd
from yahooquery import Ticker

def convert_yahooquery_to_yfinance(df_yahooquery):
    # Step 1: Reset the index to make 'Date' a regular column
    df_yahooquery.reset_index(inplace=True)

    # Step 2: Rename columns
    df_yahooquery.rename(columns={
        'date': 'Date',
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close',
        'adjclose': 'Adj Close',
        'volume': 'Volume'
    }, inplace=True)

    # Step 3: Convert "Date" to a string in "YYYY-MM-DD" format
    df_yahooquery['Date'] = pd.to_datetime(df_yahooquery['Date']).dt.strftime('%Y-%m-%d')

    # Step 4: Set "Date" as the index
    df_yahooquery.set_index('Date', inplace=True)

    # Step 5: Remove the "symbol" and "dividends" column
    df_yahooquery.drop('symbol', axis=1, inplace=True)
    df_yahooquery.drop('dividends', axis=1, inplace=True)
    return df_yahooquery

# Example usage:
# Assuming you have a DataFrame df_yahooquery from yahooquery.history

stock = Ticker("SBIN.NS")
_df = stock.history(interval='1d', 
                    start='2022-01-01', 
                    end=(dt.now() + delta(1)).strftime('%Y-%m-%d'))
print(_df)
df_yfinance = convert_yahooquery_to_yfinance(_df)
print(df_yfinance)

In [4]:
import pandas as pd
from yahooquery import Ticker

def get_stock_info(symbol):
    try:
        tickers = Ticker(symbol, validate=True)
        datasi = tickers.get_modules("summaryProfile quoteType")
        dfsi = pd.DataFrame.from_dict(datasi).T
        dataframes = [pd.json_normalize([x for x in dfsi[module] if isinstance(x, dict)]) for
                      module in ['summaryProfile', 'quoteType']]
        #print(dataframes)
        dfsi = pd.concat(dataframes, axis=1)
        
        dfsi = dfsi.set_index('symbol')
        
        return dfsi
    except Exception as e:
        #return f"An error occurred: {e}"
        return None

# Example usage:
symbol = 'CDW'  # Replace with the stock symbol you want to retrieve information for
stock_info = get_stock_info(symbol)
if isinstance(stock_info, pd.DataFrame):
    print(stock_info['industry'].iloc[0])
    print(stock_info["sector"].iloc[0])
    print("test")


Information Technology Services
Technology
test


In [ ]:
end = dt.today().strftime('%Y-%m-%d')
start = (dt.now() - delta(365)).strftime('%Y-%m-%d')

print(start)

import datetime as dt
if end is None:
    end = dt.datetime.today().strftime('%Y-%m-%d')
elif type(end) is str:
    end = dt.datetime.strptime(end, '%Y-%m-%d')
if start is None:
    start = (dt.datetime.now() - dt.timedelta(365)).strftime('%Y-%m-%d')
elif type(start) is str:
    start = dt.datetime.strptime(start, '%Y-%m-%d')

In [1]:
def remove_key(dictionary, key_to_remove):
    """
    Remove a key and its associated data from a dictionary.

    Parameters:
    - dictionary (dict): The input dictionary.
    - key_to_remove (str): The key to be removed.

    Returns:
    - dict: The modified dictionary without the specified key.
    """
    if key_to_remove in dictionary:
        del dictionary[key_to_remove]
    return dictionary

# Example usage:
my_dict = {'a': 1, 'b': 2, 'c': 3}
key_to_remove = 'b'

modified_dict = remove_key(my_dict, key_to_remove)
print(modified_dict)


{'a': 1, 'c': 3}


In [ ]:
import plotly.express as px
fig = px.bar(x=["a", "b", "c"], y=[1, 3, 2])
fig.show()

In [ ]:
    # Plotting MACD
    plt.figure(figsize=(10, 3))
    plt.bar(df.index, df['macdh_12_26_9'], label='MACD Histogram', color='gray')
    plt.plot(df.index, df['macd_12_26_9'], label='MACD', color='blue')
    plt.plot(df.index, df['macds_12_26_9'], label='Signal Line', color='orange')
    plt.axhline(0, linestyle='--', color='gray', alpha=0.5)  # Zero line
    plt.title('MACD (Moving Average Convergence Divergence)')
    plt.legend()

In [ ]:
def plot_stratagy_one(ticker, df):

    # Calculate RSI
    delta = df['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    df = df.tail(200)

    # Plotting
    fig, axes = plt.subplots(3, 1, figsize=(10, 12), sharex=True)
    plt.grid(axis='both')
    plt.yticks(fontsize=12)
    plt.xticks(rotation=45)

    # Plotting Price and SMAs
    axes[0].plot(df.index, df['Close'], label='Close Price', color='black')
    axes[0].plot(df.index, df['SMA_10'], label='SMA 10', color='green')
    axes[0].plot(df.index, df['SMA_30'], label='SMA 30', color='yellow')
    axes[0].plot(df.index, df['SMA_50'], label='SMA 50', color='orange')
    axes[0].plot(df.index, df['SMA_200'], label='SMA 200', color='blue')
    axes[0].legend()
    axes[0].set_title(f'{ticker} Price and SMAs')

    # Plotting RSI
    axes[1].plot(df.index, df['RSI'], label='RSI', color='purple')
    axes[1].axhline(70, linestyle='--', color='red', alpha=0.5)  # Overbought threshold
    axes[1].axhline(30, linestyle='--', color='green', alpha=0.5)  # Oversold threshold
    axes[1].set_title(f'{ticker} Relative Strength Index (RSI)')
    axes[1].legend()

    # Plotting MACD
    axes[2].bar(df.index, df['MACD_hist'], label='MACD Histogram', color='gray')
    axes[2].plot(df.index, df['MACD'], label='MACD', color='blue')
    axes[2].plot(df.index, df['MACD_signal'], label='Signal Line', color='orange')
    axes[2].axhline(0, linestyle='--', color='gray', alpha=0.5)  # Zero line
    axes[2].set_title(f'{ticker} MACD (Moving Average Convergence Divergence)')
    axes[2].legend()

    plt.xlabel('Date')
    plt.tight_layout()
    # Save the third figure
    plt.savefig(f'plots/{ticker}.png', dpi=fig.dpi)

In [ ]:
def plot_strategy_multiple(tickers, data, max_bars=200):
    """
    Plot RSI, Price and SMAs, and MACD for multiple stocks in a single image.

    Parameters:
    - tickers: list, a list of stock ticker symbols
    - data: dict, containing stock data for each ticker
    - max_bars: int, maximum number of bars to display on the x-axis
    """

    num_stocks = len(tickers)
    num_columns = 3
    j = 0
    fig = plt.figure(figsize=(20, max(num_stocks, 5)), dpi=200)
    gs = gridspec.GridSpec(int(np.ceil(num_stocks / num_columns)), num_columns)

    # Calculate RSI, Price, SMAs, and MACD for each stock
    for i, ticker in enumerate(tickers):
        df = data[ticker]

        # Calculate RSI
        delta = df['Close'].diff(1)
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(window=14).mean()
        avg_loss = loss.rolling(window=14).mean()
        rs = avg_gain / avg_loss
        df['RSI'] = 100 - (100 / (1 + rs))
        df = df.tail(200)

        # Create subplots using gridspec
        j += 1
        ax1 = plt.subplot(gs[i])
        ax2 = ax1.twinx()  # Create a twin Axes sharing the xaxis

        # Plotting RSI
        ax1.plot(df['RSI'], label='RSI', color='purple')
        ax1.axhline(70, linestyle='--', color='red', alpha=0.5)  # Overbought threshold
        ax1.axhline(30, linestyle='--', color='green', alpha=0.5)  # Oversold threshold
        ax1.axhline(50, linestyle='--', color='black', alpha=0.5)
        ax1.tick_params(axis='y', labelsize=12)
        ax1.legend(loc="upper left")

        # Plotting Price and SMAs
        ax2.plot(df['Close'], label='Close Price', color='black')
        ax2.plot(df['SMA_10'], label='SMA 10', color='green')
        ax2.plot(df['SMA_30'], label='SMA 30', color='yellow')
        ax2.plot(df['SMA_50'], label='SMA 50', color='orange')
        ax2.plot(df['SMA_200'], label='SMA 200', color='blue')
        ax2.tick_params(axis='y', labelsize=12)
        ax2.legend(loc="upper right")

        # Customize x-axis date labels
        ax1.xaxis.set_major_locator(plt.MaxNLocator(max_bars // 10))  # Show every 10th date
        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))  # Format date labels
        ax1.tick_params(axis='x', rotation=45)

        # Adjust space between subplots
        plt.subplots_adjust(hspace=0.5)

    # Set overall title for the entire plot
    plt.suptitle('Multiple Stock Analysis')

    # Save the figure with a unique name
    save_dir = 'plots'
    os.makedirs(save_dir, exist_ok=True)
    fig.savefig(f'{save_dir}/plot_strategy_multiple.png', dpi=fig.dpi)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_stock_indicators(data, max_bars=200):
    fig, axs = plt.subplots(3, 1, figsize=(10, 8), sharex=True)

    # Plot SMA
    axs[0].plot(data['Close'], label='Close Price')
    axs[0].plot(data['Close'].rolling(window=50).mean(), label='SMA 50', linestyle='--')
    axs[0].plot(data['Close'].rolling(window=200).mean(), label='SMA 200', linestyle='--')
    axs[0].legend()
    axs[0].set_title('Simple Moving Averages (SMA)')

    # Plot RSI
    axs[1].plot(data['RSI'], label='RSI')
    axs[1].axhline(y=70, color='r', linestyle='--', label='Overbought (70)')
    axs[1].axhline(y=30, color='g', linestyle='--', label='Oversold (30)')
    axs[1].legend()
    axs[1].set_title('Relative Strength Index (RSI)')

    # Plot MACD
    axs[2].plot(data['MACD'], label='MACD')
    axs[2].plot(data['Signal_Line'], label='Signal Line', linestyle='--')
    axs[2].bar(data.index, data['MACD_Histogram'], color='gray', alpha=0.5, label='MACD Histogram')
    axs[2].legend()
    axs[2].set_title('Moving Average Convergence Divergence (MACD)')

    # Customize x-axis date labels
    axs[2].xaxis.set_major_locator(plt.MaxNLocator(max_bars // 10))  # Show every 10th date
    axs[2].xaxis.set_major_formatter(plt.DateFormatter('%Y-%m-%d'))  # Format date labels
    plt.xticks(rotation=45)  # Rotate date labels for better visibility

    plt.xlabel('Date')
    plt.tight_layout()
    plt.show()

# Example usage
# Assuming data contains 'Close', 'RSI', 'MACD', 'Signal_Line', and 'MACD_Histogram' columns
# plot_stock_indicators(data)
